In [1]:
import time
import os
import csv

import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

In [2]:
#!pwd

In [3]:
#os.chdir('C:/Users/matth/Documents/Coding/RA Fouirnaies/vs_candidate_bios')

## Candidate Biographies

### Exploring Candidate Pages

Candidate bio page follows this https://justfacts.votesmart.org/candidate/biography/333/janet-carroll-richardson where the name gets automatically filled after inputting the number

Appears to end at candidate 209,867. However, there could be gaps in the middle.

In [4]:
def make_query(num):
    query = 'https://justfacts.votesmart.org/candidate/biography/' + str(num)
    return query

**Initial Remarks**: The biographical information of candidates is organized into a number of collapsible cards ['Personal','Education','Political Experience', 'Caucuses/Former Committees', 'Professional Experience', 'Religious, Civic, and other Memberships'] (6 total). Information within the first card 'Personal' is organized slightly differently than in the other cards, with a bold header for each piece of information, while in the other 5 cards information is presented plainly in rows. 

The current plan is to create a general function or functions which loop through the individual attributes of each card for every card. 

In [5]:

q = make_query(333)
request = requests.get(q)
soup = BeautifulSoup(request.content, 'html.parser')


In [6]:
cards = soup.find_all(name = 'div', class_ = 'card card-plain accordion-card')

In [7]:
len(cards) # all cards besides education card which has class 'card card-plain accordion-card accordion-header'

5

**Personal**

In [8]:
vals = cards[0].find_all(name = 'p')
len(vals)

12

In [9]:
for val in vals:
    print(val.string.strip())

Full Name:
Janet Carroll 'Skeet' Richardson
Gender:
Female
Family:
Divorced;   1 daughter: Bethany, 10 yrs. old
Birth Date:
01/09/1959
Birth Place:
Fort Worth, TX
Home City:
Keller, TX


In [10]:
entries = []
for x in range(0,len(vals),2):
    if(x==(len(vals)-2)):
        entry = vals[x].string.strip() + vals[x+1].string.strip()
    else:
        entry = vals[x].string.strip() + vals[x+1].string.strip() + ';'
    entries.append(entry)

In [11]:
entries # could improve this method by creating a dictionary instead of a list

["Full Name:Janet Carroll 'Skeet' Richardson;",
 'Gender:Female;',
 'Family:Divorced;   1 daughter: Bethany, 10 yrs. old;',
 'Birth Date:01/09/1959;',
 'Birth Place:Fort Worth, TX;',
 'Home City:Keller, TX']

**Education**

In [12]:
edu_card = soup.find(name = 'div', class_ = 'card card-plain accordion-card accordion-header')

In [13]:
edu_list = edu_card.find_all(name = 'p')

In [14]:
for edu in edu_list:
    print(edu.string.strip())

Texas Christian University


**Others**

In [15]:
others = []
for card in cards:
    if(card != cards[0]):
        temp = card.find_all(name = 'p')
        o = []
        for t in temp:
            o.append(t.string.strip())
        others.append(o)

In [16]:
others

[['Legislative Aide, State Representative Doyle Willis',
  'Campaign Director for Congressional candidate George Richardson, 1986'],
 ['No caucus information on file.'],
 ["President/Producer, `Positive' TV Programming, Richardson-Carroll Prod",
  "Author, `Unsung Heroes'",
  'Public Relations Executive Television Executive, WFAA-TV, ABC'],
 ["Regional Member, Emmy's",
  'Member, National Association of Female Executives',
  'Member, Women in Communications']]

#### Testing Function(s) for scraping an individual page and writing it to a row of a csv

In [17]:
from cand_methods import scrape_candbio

In [18]:
headers = ['Candidate Number','Personal','Education','Political Experience','Caucuses/Former Committees',
           'Professional Experience','Religious, Civic, and other Memberships','Additional Information']

In [19]:
file = open('./draft_cbios.csv', 'w')
writer = csv.writer(file)
writer.writerow(headers)
file.close()

In [20]:
start_time = time.time()

#for x in range(1,101):
#    scrape_candbio(x)


    
print("My program took", time.time()-start_time, "to run") # 467.9851350784302 (8 min) for 100 bios

My program took 467.9851350784302 to run


**Troubleshooting**

In [1]:
from cand_methods import make_query
from cand_methods import get_personal
from cand_methods import get_edu
from cand_methods import get_others

In [62]:
num = 73
query = make_query(num)   
try:
    req = requests.get(query)
except:
    row = [num] + ['redirect error' for i in range(0,6)]

try:
    soup = BeautifulSoup(req.content, 'html.parser')
except:
    row = [num] + ['unknown' for i in range(0,6)]

# checking if page is missing/empty i.e. there is no candidate with that id number
try:
    main = soup.find(name='div', class_ = 'row text-center')
    cont = main.find(name = 'div', class_ = 'container')
    pnf = cont.find(name='h1', class_ = 'title')
    if (pnf.string == 'Page Not Found'):
        row = [num] + [np.nan for i in range(0,6)]
except:
    pass

cards = soup.find_all(name = 'div', class_ = 'card card-plain accordion-card') # all cards besides education
edu_card = soup.find(name = 'div', class_ = 'card card-plain accordion-card accordion-header')

### Section